In [1]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete (Trial with index options)
# Run-time: Unknown

# Dependencies:
#  - List of indexes and equities
#  - underlying's ohlc history for standard deviation and Black Scholes

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=5) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=5) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=5) # kavi IBG paper-trade

In [166]:
#***      Imports    *****
import pandas as pd
from itertools import product

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

#....... get option chains for nse
df_s = pd.read_pickle(datapath+'df_nse_idx_symbols.pkl')

symbols = [Index(symbol=s, exchange='NSE') for s in df_s.ibSymbol]

qual_symbols = ib.qualifyContracts(*symbols)

chains = {s: ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in qual_symbols}

tup_chains = [(k.symbol, i.underlyingConId, 
               tuple(i.expirations), tuple(i.strikes)) 
 for k, v in chains.items() 
 for i in v]

#...... qualify options and strikes

rights=['P'] # currently taking puts only

L = [[[x[0]], [x[1]], x[2], x[3], rights] for x in tup_chains]
df1 = pd.DataFrame([j for i in L for j in product(*i)], 
             columns=['ibSymbol', 'undConId', 'expiry', 'strike', 'right'])

In [167]:
df = df1[:3] # !!!!DATA LIMITER!!!

options = [Option(symbol=s,  lastTradeDateOrContractMonth=e, strike=k, right=r, exchange='NSE')
for s in df.ibSymbol
for e in df.expiry
for k in df.strike
for r in df.right]

qual_options = ib.qualifyContracts(*options)

In [175]:
df

,ibSymbol,undConId,expiry,strike,right
0,BANKNIFTY,56994300,20190131,25600.0,P
1,BANKNIFTY,56994300,20190131,23300.0,P
2,BANKNIFTY,56994300,20190131,26500.0,P


In [174]:
qual_options

[Option(conId=339376491, symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190131', strike=25600.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='BANKNIFTY19JAN25600PE', tradingClass='BANKNIFTY'),
 Option(conId=339376491, symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190131', strike=25600.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='BANKNIFTY19JAN25600PE', tradingClass='BANKNIFTY'),
 Option(conId=339376491, symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190131', strike=25600.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='BANKNIFTY19JAN25600PE', tradingClass='BANKNIFTY'),
 Option(conId=339376306, symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190131', strike=23300.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='BANKNIFTY19JAN23300PE', tradingClass='BANKNIFTY'),
 Option(conId=339376306, symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190131', strike=23300.0

# To-do
* something is wrong in the qual_options. Fix it.
* integration with standard deviation
* remove unnecessary options (Put)
* integration with black scholes
* integration with lot and margin
* generate orders
* place orders (what-if) to check margins (duplicate)
* check latest prices
* screen most valueable RoM and PoP
* place orders